In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Load data

We load to alternative tables as exlained in the notebook '0-data-preparation'. 

In [2]:
btvote = pd.read_pickle('../data/btvote.pkl')
btvote.head()

,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,yes
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,yes,yes,yes,yes,yes,yes,no,absence,absence,absence
2,Linke,no,no,no,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,abstain,no,no
3,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,absence
4,Linke,absence,absence,absence,absence,absence,absence,absence,absence,absence,...,no,no,no,no,no,no,no,abstain,no,absence


In [3]:
btvote_alt = pd.read_pickle('../data/btvote_alternative.pkl')
btvote_alt.head()

,party,vote_19001,vote_19002,vote_19003,vote_19004,vote_19005,vote_19006,vote_19007,vote_19008,vote_19009,...,vote_19235,vote_19236,vote_19237,vote_19238,vote_19239,vote_19240,vote_19241,vote_19242,vote_19243,vote_19244
0,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,yes
1,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,yes,yes,yes,yes,yes,yes,no,excused absence,excused absence,unexcused absent
2,Linke,no,no,no,no,no,no,no,no,yes,...,no,no,no,no,no,no,no,abstain,no,no
3,CDU/CSU,yes,yes,yes,yes,yes,yes,yes,yes,no,...,yes,yes,yes,yes,yes,yes,no,yes,yes,excused absence
4,Linke,excused absence,excused absence,excused absence,excused absence,excused absence,unexcused absent,unexcused absent,unexcused absent,excused absence,...,no,no,no,no,no,no,no,abstain,no,excused absence


# Split data and encode target variable

In [4]:
# Split dataframe in 'data' and 'target'
btvote_data = btvote.drop('party', axis=1)
btvote_target = btvote['party']

# Create a 'data' part for the alternative dataset btvote_alt
btvote_data_alt = btvote_alt.drop('party', axis=1)

# Encode the target variable
label_encoder = preprocessing.LabelEncoder()
btvote_target = label_encoder.fit_transform(btvote_target)

# Pipeline and GridSearch setup

In the pipeline we include the SimpleImputer with different strategies as well as the KNNImputer with number of neighbors between 2 and 9.\
For Balancing, we just the RandomOverSampler at the moment. In a later notebook, the Balancing will be evaluated in detail.\
As an estimator, we consider nearest-neighbor, Decision Tree and Naive Bayes. Again, later more models will be analysed in detail. For the moment, this selection of estimators should just ensure well-founded results for the different Imputing methods

In [63]:
from imblearn.pipeline import Pipeline
# normalisation
from sklearn.preprocessing import OneHotEncoder
# imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
# balancing
from imblearn.over_sampling import RandomOverSampler
# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Pipeline
pipeline = Pipeline([
    ('imputer', None),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ('balancing', RandomOverSampler()),
    ('estimator', None)
])

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# specify the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# define the scoring function
# Note: As we use Balancing the micro average will equal the macro average
f1 = make_scorer(f1_score, average='micro')

# NaN handling

### Data Enrichment
Definition of NaN
1. Only actual NaNs are considered as NaNs, 'abstain' and 'absence' both as separate values\
    a. 'unexcused absent' and 'excused absence' combined as 'absence'\
    b. 'unexcused absent' and 'excused absence' as separate values
2. Consider actual NaNs and 'absence' as NaN, keep 'abstain' as possible value\
    a. encode 'no', 'abstain' and 'yes' using OneHotEncoder in pipeline\
    b. encode 'no', 'abstain' and 'yes' ordinal
3. Consider only 'yes' and 'no' as allowed values

Imputing of missing values (NaN)

- sklearn [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) with strategies *mean*, *most_frequent* and *constant*
- sklearn [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer) with different *n_neighbors*

*Limitation:* The KNNImputer and the SimpleImputer with strategy 'mean' can not deal with categorial values. If we encode the data using OneHotEncoding before, we would result in an extra column for NaN. The only option is to use an OrdinalEncoding before Imputing. In that case, the ordinal encoding influences distances and averages. So, those two Imputing approaches can only be used in experiment 2b and 3, as we cannot define an order for experiment 1.\
*Exception:* KNNImputer with n_neighbors=1 can be used in all experiments, as no averaging is performed.

### 1. 'abstain' and 'absence' as separate values
#### a. 'unexcused absent' and 'excused absence' combined as 'absence'

In [64]:
# define parameter grid
parameters = {
    'imputer': [SimpleImputer(strategy='most_frequent'), SimpleImputer(strategy='constant'), KNNImputer(n_neighbors=1)],
    'estimator': [KNeighborsClassifier(n_neighbors=7), DecisionTreeClassifier(max_depth=5, random_state=42), GaussianNB()],
}

In [66]:
# encode data
btvote_data_1a = btvote_data.replace({'no':0, 'yes':1, 'abstain':2, 'absence':3})

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data_1a, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# pivot the results for better visualization
results['param_imputer'] = results['param_imputer'].astype(str)
results['param_estimator'] = results['param_estimator'].astype(str)
pivoted_results = results.pivot(index='param_imputer', columns='param_estimator', values='mean_test_score')
pivoted_results['Average'] = pivoted_results[['DecisionTreeClassifier(max_depth=5, random_state=42)','GaussianNB()','KNeighborsClassifier(n_neighbors=7)']].mean(axis=1)
display(pivoted_results)

param_estimator,"DecisionTreeClassifier(max_depth=5, random_state=42)",GaussianNB(),KNeighborsClassifier(n_neighbors=7),Average
param_imputer,,,,
KNNImputer(n_neighbors=1),0.752901,0.809514,0.708793,0.757069
SimpleImputer(strategy='constant'),0.647063,0.738324,0.707459,0.697616
SimpleImputer(strategy='most_frequent'),0.715532,0.785243,0.730198,0.743658


We see, that on average, the KNNImputer with 'n_neighbors'=1 performs the best, then the SimpleImputer with strategy 'most_frequent' and then 'constant'. We won't go into detail on the different models, as they will be discussed in later notebooks.

#### b: 'unexcused absent' and 'excused absence' as separate values

In [67]:
# encode data
btvote_data_1b = btvote_data_alt.replace({'no':0, 'yes':1, 'abstain':2, 'unexcused absent':3, 'excused absence':4})

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data_1b, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# pivot the results for better visualization
results['param_imputer'] = results['param_imputer'].astype(str)
results['param_estimator'] = results['param_estimator'].astype(str)
pivoted_results = results.pivot(index='param_imputer', columns='param_estimator', values='mean_test_score')
pivoted_results['Average'] = pivoted_results[['DecisionTreeClassifier(max_depth=5, random_state=42)','GaussianNB()','KNeighborsClassifier(n_neighbors=7)']].mean(axis=1)
display(pivoted_results)

param_estimator,"DecisionTreeClassifier(max_depth=5, random_state=42)",GaussianNB(),KNeighborsClassifier(n_neighbors=7),Average
param_imputer,,,,
KNNImputer(n_neighbors=1),0.740847,0.804054,0.712829,0.752577
SimpleImputer(strategy='constant'),0.690036,0.740991,0.706126,0.712384
SimpleImputer(strategy='most_frequent'),0.716973,0.793297,0.712775,0.741015


We see that splitting up 'absence' into 'unexcused absent' and 'excused absence' will improve the test results slightly if anything. Having in mind that for our Wahl-O-Mat, there will be only one option for 'absence', from now on, we will use 'unexcused absent' and 'excused absence' combined as absence.

### 2. 'absence' is NaN, 'abstain' as allowed value

#### a. encode 'no', 'abstain' and 'yes' using OneHotEncoder in pipeline

The evaluation works the same as for the case before. We just convert all 'absence' values to NaN before executing the grid search.

In [68]:
# encode data
btvote_data_2a = btvote_data.replace({'absence':np.nan, 'no':0, 'yes':1, 'abstain':2})

# create the grid search instance
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data_2a, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# pivot the results for better visualization
results['param_imputer'] = results['param_imputer'].astype(str)
results['param_estimator'] = results['param_estimator'].astype(str)
pivoted_results = results.pivot(index='param_imputer', columns='param_estimator', values='mean_test_score')
pivoted_results['Average'] = pivoted_results[['DecisionTreeClassifier(max_depth=5, random_state=42)','GaussianNB()','KNeighborsClassifier(n_neighbors=7)']].mean(axis=1)
display(pivoted_results)

param_estimator,"DecisionTreeClassifier(max_depth=5, random_state=42)",GaussianNB(),KNeighborsClassifier(n_neighbors=7),Average
param_imputer,,,,
KNNImputer(n_neighbors=1),0.771658,0.814811,0.795910,0.794126
SimpleImputer(strategy='constant'),0.710234,0.707423,0.726324,0.714661
SimpleImputer(strategy='most_frequent'),0.691369,0.778649,0.770450,0.746823


Removing 'absence' as value significantly improves performance for the KNNImputer. For the other two Imputers the results are similar to the previous experiments. Overall, we can say, that using 'absence' as information reduces performance.

#### b. encode 'no', 'abstain' and 'yes' ordinal

Now we define a new pipeline without the OneHotEncoder. We encode the ordinal before executing the grid search: {'no':0}<{'abstain':0.5}<{'yes':1}
And now, these ordinal encodings can be used for classification and imputing. 
In this case, all imputers listed above can be used, as the voting behavior is in numeric format. That is why we redefine the parameter grid.

In [70]:
# ordinal pipeline without OneHotEncoder
ordinal_pipeline = Pipeline([('imputer', None), ('balancing', RandomOverSampler()), ('estimator', None)])

In [69]:
# redefine parameter grid
ordinal_parameters = [
    {
        'imputer': [SimpleImputer(strategy='mean'), SimpleImputer(strategy='most_frequent'), SimpleImputer(strategy='constant')],
        'estimator': [KNeighborsClassifier(n_neighbors=7), DecisionTreeClassifier(max_depth=5, random_state=42), GaussianNB()],
    }, {
        'imputer': [KNNImputer()],
        'imputer__n_neighbors': range(1,10),
        'estimator': [KNeighborsClassifier(n_neighbors=7), DecisionTreeClassifier(max_depth=5, random_state=42), GaussianNB()],
    }
]

In [71]:
# encode data
btvote_data_2b = btvote_data.replace({'absence':np.nan, 'no':0, 'abstain':0.5, 'yes':1})

# create the grid search instance
grid_search_estimator = GridSearchCV(ordinal_pipeline, ordinal_parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data_2b, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# pivot the results for better visualization
results['param_imputer'] = results['param_imputer'].astype(str)
results['param_estimator'] = results['param_estimator'].astype(str)
pivoted_results = results.pivot(index=['param_imputer','param_imputer__n_neighbors'], columns='param_estimator', values='mean_test_score')
pivoted_results['Average'] = pivoted_results[['DecisionTreeClassifier(max_depth=5, random_state=42)','GaussianNB()','KNeighborsClassifier(n_neighbors=7)']].mean(axis=1)
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_29716\3005397640.py:16: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index=['param_imputer','param_imputer__n_neighbors'], columns='param_estimator', values='mean_test_score')


param_estimator                                                     DecisionTreeClassifier(max_depth=5, random_state=42)  \
param_imputer                           param_imputer__n_neighbors                                                         
KNNImputer()                            1                                                                    0.779856      
                                        2                                                                    0.774486      
                                        3                                                                    0.771802      
                                        4                                                                    0.774414      
                                        5                                                                    0.781225      
                                        6                                                                    0.778559      
                                        7                                                                    0.787982      
                                        8                                                                    0.775784      
                                        9                                                                    0.774559      
SimpleImputer()                         NaN                                                                  0.679297      
SimpleImputer(strategy='constant')      NaN                                                                  0.722090      
SimpleImputer(strategy='most_frequent') NaN                                                                  0.680523      

param_estimator                                                     GaussianNB()  \
param_imputer                           param_imputer__n_neighbors                 
KNNImputer()                            1                               0.810757   
                                        2                               0.790739   
                                        3                               0.757297   
                                        4                               0.703568   
                                        5                               0.699514   
                                        6                               0.699477   
                                        7                               0.708955   
                                        8                               0.734468   
                                        9                               0.756000   
SimpleImputer()                         NaN                             0.817459   
SimpleImputer(strategy='constant')      NaN                             0.665838   
SimpleImputer(strategy='most_frequent') NaN                             0.790667   

param_estimator                                                     KNeighborsClassifier(n_neighbors=7)  \
param_imputer                           param_imputer__n_neighbors                                        
KNNImputer()                            1                                                      0.802613   
                                        2                                                      0.797279   
                                        3                                                      0.789225   
                                        4                                                      0.796072   
                                        5                                                      0.800018   
                                        6                                                      0.794685   
                                        7                                                      0.793387   
                                        8                                                      0.789333   
                

Encoding the values 'no', 'abstain' and 'yes' ordinal results in similar values for the methods that could be applied before. As for we saved the OneHotEncoding step by directly using this ordinal encoding, this strategy is more desirable.

### 3. only 'yes' and 'no' are allowed values

Again, all imputers listed above can be used, as the voting behavior is in numeric format.

In [73]:
# encode data
btvote_data_3 = btvote_data.replace({'absence':np.nan, 'abstain':np.nan, 'no':0, 'yes':1})

# create the grid search instance
grid_search_estimator = GridSearchCV(ordinal_pipeline, ordinal_parameters, scoring=f1, cv=stratified_10_fold_cv, error_score='raise')

# run the grid search
grid_search_estimator.fit(btvote_data_3, btvote_target)

# results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)

# pivot the results for better visualization
results['param_imputer'] = results['param_imputer'].astype(str)
results['param_estimator'] = results['param_estimator'].astype(str)
pivoted_results = results.pivot(index=['param_imputer','param_imputer__n_neighbors'], columns='param_estimator', values='mean_test_score')
pivoted_results['Average'] = pivoted_results[['DecisionTreeClassifier(max_depth=5, random_state=42)','GaussianNB()','KNeighborsClassifier(n_neighbors=7)']].mean(axis=1)
display(pivoted_results)

C:\Users\mogra\AppData\Local\Temp\ipykernel_29716\3633095267.py:16: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivoted_results = results.pivot(index=['param_imputer','param_imputer__n_neighbors'], columns='param_estimator', values='mean_test_score')


param_estimator                                                     DecisionTreeClassifier(max_depth=5, random_state=42)  \
param_imputer                           param_imputer__n_neighbors                                                         
KNNImputer(n_neighbors=1)               1                                                                    0.782486      
                                        2                                                                    0.769135      
                                        3                                                                    0.778523      
                                        4                                                                    0.785153      
                                        5                                                                    0.783928      
                                        6                                                                    0.775892      
                                        7                                                                    0.775838      
                                        8                                                                    0.781243      
                                        9                                                                    0.767712      
SimpleImputer()                         NaN                                                                  0.714234      
SimpleImputer(strategy='constant')      NaN                                                                  0.694036      
SimpleImputer(strategy='most_frequent') NaN                                                                  0.701982      

param_estimator                                                     GaussianNB()  \
param_imputer                           param_imputer__n_neighbors                 
KNNImputer(n_neighbors=1)               1                               0.816198   
                                        2                               0.802811   
                                        3                               0.789387   
                                        4                               0.796144   
                                        5                               0.800108   
                                        6                               0.798739   
                                        7                               0.796054   
                                        8                               0.798739   
                                        9                               0.796054   
SimpleImputer()                         NaN                             0.802775   
SimpleImputer(strategy='constant')      NaN                             0.711820   
SimpleImputer(strategy='most_frequent') NaN                             0.793333   

param_estimator                                                     KNeighborsClassifier(n_neighbors=7)  \
param_imputer                           param_imputer__n_neighbors                                        
KNNImputer(n_neighbors=1)               1                                                      0.785189   
                                        2                                                      0.787892   
                                        3                                                      0.786523   
                                        4                                                      0.789279   
                                        5                                                      0.788000   
                                        6                                                      0.781315   
                                        7                                                      0.783910   
                                        8                                                      0.783928   
                

Compared to the previous experiment, the results for the KNNImputer are more stable for larger values of n_neighbors. The SimpleImputers perform similar to the previous experiment.

# Conclusion
We've seen that generally the performance is higher, when using KNNImputer instead of the SimpleImputer independent of the strategy used. The highest F1 score can be achieved by setting n_neighbors to 1, so that only the nearest neighbor is considered. So from now on we will use this as our imputer.

We simultaniously evaluated different input formats. We got a clear result, that using 'abstain' and 'absence' as possible values for the input data generally leads to a lower F1-Score. Whether to consider 'abstain' as value doesn't really influence the results. As we are interested in using an 'abstain' option for our model later, we will keep this value.\
Using an OneHotEncoder instead of an OrdinalEncoder doesn't improve the results and just adds a computation overhead. We will finally use option 2b in the future experiments.

# Dataset generation

We will finally generate and export the encoded dataset for option 2b, so we can just import the encoded dataset in the next experiments.

In [74]:
btvote_encoded = btvote.replace({'absence':np.nan, 'no':0, 'abstain':0.5, 'yes':1})
btvote_encoded.to_pickle('../data/btvote_encoded.pkl')

# Archive
GroupImputer

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a custom Imputer that imputes based on the party
class GroupImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean'):
        self.strategy = strategy

    def fit(self, X, y):
        # store the y series to find the correct index in the transform function
        self.y = y
        # self.group_values becomes an attribute that is the map of group_specific values
        if self.strategy == 'mode':
            self.group_values = X.groupby(self.y).agg(lambda x: pd.Series.mode(x)[0])
        else:
            self.group_values = X.groupby(self.y).mean()
        return self

    def transform(self, X):
        for column in X.columns:
            if X[column].isna().any():
                # apply to all columns that contain any NaN values:
                # if the current cell in the DataFrame is a NaN value, return the group-specific value for that cell
                # othwerwise return the original value
                X[column] = X.apply(
                    lambda row: self.group_values.loc[self.y[row.name],column] if np.isnan(row[column])
                                   else row[column], axis=1)
        return X.to_numpy()